In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
# Add the channel dimension (LeNet expects 3 dimensions: Height, Width, Channel)
# MNIST comes as (60000, 28, 28), we need (60000, 28, 28, 1)

x_train = tf.expand_dims(x_train, axis=-1)
x_test = tf.expand_dims(x_test, axis=-1)

In [4]:
# Resize images from 28x28 to 32x32 (Required for strict LeNet-5)
x_train = tf.image.resize(x_train, [32, 32])
x_test = tf.image.resize(x_test, [32, 32])

# Normalize pixel values to range 0-1
x_train = x_train / 255.0
x_test = x_test / 255.0

print(f"Training shape: {x_train.shape}")  # Output: (60000, 32, 32, 1)

Training shape: (60000, 32, 32, 1)


In [5]:
from tensorflow.keras import layers, models, Input

def build_lenet5(input_shape=(32, 32, 1), num_classes=10):
    
    inputs = Input(shape=input_shape)

    # Layer 1: Convolutional Layer (C1)
    # 6 filters, 5x5 kernel, tanh activation
    # Padding='valid' reduces 32x32 -> 28x28
    x = layers.Conv2D(filters=6, kernel_size=(5, 5), strides=1, activation='tanh', padding='valid', name='C1')(inputs)

    # Layer 2: Average Pooling Layer (S2)
    # 2x2 pool, stride 2
    # Reduces 28x28 -> 14x14
    x = layers.AveragePooling2D(pool_size=(2, 2), strides=2, name='S2')(x)

    # Layer 3: Convolutional Layer (C3)
    # 16 filters, 5x5 kernel, tanh activation
    # Padding='valid' reduces 14x14 -> 10x10
    x = layers.Conv2D(filters=16, kernel_size=(5, 5), strides=1, activation='tanh', padding='valid', name='C3')(x)

    # Layer 4: Average Pooling Layer (S4)
    # 2x2 pool, stride 2
    # Reduces 10x10 -> 5x5
    x = layers.AveragePooling2D(pool_size=(2, 2), strides=2, name='S4')(x)

    # Flatten the output to feed into Dense layers
    x = layers.Flatten()(x)

    # Layer 5: Fully Connected / Convolutional (C5)
    # 120 units, tanh activation
    x = layers.Dense(units=120, activation='tanh', name='C5')(x)

    # Layer 6: Fully Connected (F6)
    # 84 units, tanh activation
    x = layers.Dense(units=84, activation='tanh', name='F6')(x)

    # Output Layer
    # Softmax activation for classification probabilities
    outputs = layers.Dense(units=num_classes, activation='softmax', name='Output')(x)

    # Create the model
    model = models.Model(inputs=inputs, outputs=outputs, name='LeNet-5')
    
    return model

In [6]:
lenet_model = build_lenet5(input_shape=(32, 32, 1), num_classes=10)

In [7]:
lenet_model.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

In [8]:
lenet_model.summary()

Model: "LeNet-5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 32, 32, 1)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ C1 (Conv2D)                          │ (None, 28, 28, 6)           │             156 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ S2 (AveragePooling2D)                │ (None, 14, 14, 6)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ C3 (Conv2D)                          │ (None, 10, 10, 16)          │           2,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ S4 (AveragePooling2D)                │ (None, 5, 5, 16)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 400)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ C5 (Dense)                           │ (None, 120)                 │          48,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ F6 (Dense)                           │ (None, 84)                  │          10,164 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Output (Dense)                       │ (None, 10)                  │             850 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 61,706 (241.04 KB)

 Trainable params: 61,706 (241.04 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
history = lenet_model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 19s 9ms/step - accuracy: 0.9343 - loss: 0.2172 - val_accuracy: 0.9642 - val_loss: 0.1144
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.9750 - loss: 0.0842 - val_accuracy: 0.9791 - val_loss: 0.0685
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.9810 - loss: 0.0605 - val_accuracy: 0.9831 - val_loss: 0.0564
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.9854 - loss: 0.0458 - val_accuracy: 0.9854 - val_loss: 0.0465
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.9882 - loss: 0.0367 - val_accuracy: 0.9792 - val_loss: 0.0645


# LeNet-5 Implementation: Functional API vs Sequential API

## 📋 Overview

This LeNet-5 implementation uses the **Keras Functional API** (`x = layer(x)`) rather than the Sequential API (`model.add(layer)`). This is a deliberate architectural choice that prioritizes flexibility, scalability, and production readiness.

---

## 🔍 What is `x = layer(x)`?

When you see code like this:
```python
x = layers.Conv2D(6, kernel_size=(5, 5), activation='tanh')(inputs)
x = layers.AveragePooling2D(pool_size=(2, 2))(x)
x = layers.Conv2D(16, kernel_size=(5, 5), activation='tanh')(x)
```

Each line explicitly passes the output tensor from one layer to the next. The variable `x` represents the evolving feature tensor as it flows through the network.

**Conceptual Flow:**
```
inputs → C1 → S2 → C3 → S4 → Flatten → C5 → F6 → Output
```

This builds a **computational graph**, not just a linear list of layers.

---

## ⚖️ Sequential API vs Functional API

### Sequential API Example
```python
model = Sequential()
model.add(Conv2D(6, (5, 5), activation='tanh', input_shape=(32, 32, 1)))
model.add(AveragePooling2D((2, 2)))
model.add(Conv2D(16, (5, 5), activation='tanh'))
# ... continues linearly
```

### Functional API Example
```python
inputs = layers.Input(shape=(32, 32, 1))
x = layers.Conv2D(6, (5, 5), activation='tanh')(inputs)
x = layers.AveragePooling2D((2, 2))(x)
x = layers.Conv2D(16, (5, 5), activation='tanh')(x)
# ... continues with explicit connections
model = Model(inputs, outputs)
```

---

## ✅ Why Use the Functional API?

### 1. **Architectural Flexibility**

The Functional API supports advanced patterns that Sequential cannot handle:

- **Skip connections** (ResNet-style)
- **Multi-branch networks** (Inception-style)
- **Shared layers** (Siamese networks)
- **Multiple inputs/outputs**

### 2. **Explicit Input-Output Contract**
```python
model = Model(inputs, outputs)
```

This clearly defines what enters and exits the model, which is critical for:
- Production pipelines
- Transfer learning
- Model explainability

### 3. **Easy Access to Intermediate Layers**

Extract features from any layer without rebuilding:
```python
intermediate_model = Model(inputs, model.get_layer('C3').output)
features = intermediate_model.predict(data)
```

### 4. **Future-Proof Design**

Even though LeNet-5 is linear, starting with the Functional API:
- Eliminates future refactoring when adding complexity
- Aligns with modern deep learning best practices
- Enables seamless architectural extensions

---

## 📊 Comparison Table

| Aspect | Sequential API | Functional API |
|--------|---------------|----------------|
| **Architecture** | Linear only | Graph-based |
| **Multi-input/output** | ❌ No | ✅ Yes |
| **Skip connections** | ❌ No | ✅ Yes |
| **Intermediate outputs** | ❌ Limited | ✅ Easy |
| **Production readiness** | Medium | High |
| **Code complexity** | Lower (simple cases) | Slightly higher |
| **Flexibility** | Low | High |

---

## 🤔 Why the Variable Name `x`?

`x` is a widely adopted convention in deep learning to represent the current tensor flowing through the network. It:
- Mirrors research papers and architecture diagrams
- Keeps code concise and readable
- Is universally recognized in the ML community

You could use any name (`output`, `features`, `tensor`), but `x` is the standard.

---

## 💡 Key Takeaway

Using `x = layer(x)` isn't just syntax—it's a **strategic modeling choice**. The Functional API provides:

✅ Full control over the computation graph  
✅ Extensibility for complex architectures  
✅ Production-ready code structure  
✅ Alignment with industry best practices

Even for simple architectures like LeNet-5, the Functional API sets you up for success as your models grow in complexity.